## Práctica con data de sondajes (Limpieza)

In [ ]:
import pandas as pd

## COLLAR

In [ ]:
collar = pd.read_csv("files/dh_raw/WP_collar.csv")

In [ ]:
collar.columns

In [ ]:
collar.dtypes

- BHID: código de cada sondaje (string)
- XCOLLAR, YCOLLAR, ZCOLLAR: coordenadas del collar de cada sondaje (float)
- AT: longitud del sondaje (float)

In [ ]:
# Verificar valores vacíos
collar.isna().sum(axis=0)

In [ ]:
# Valores únicos de BHID
collar["BHID"].unique()

In [ ]:
len(collar["BHID"].unique())

- Hay 3 sondajes que faltan dentro de los códigos (WP001 a WP058)

In [ ]:
len(collar)

In [ ]:
collar.head()

In [ ]:
# Si XCOLLAR fuera una columna de string
collar["XCOLLAR"] = collar["XCOLLAR"].astype(float)

## SURVEY

In [ ]:
survey = pd.read_csv("files/dh_raw/WP_survey.csv")

In [ ]:
collar.head()

In [ ]:
survey.head()

In [ ]:
survey.info()

- BHID: el código de cada sondaje (string)
- AT: longitud en la cual se tomó el registro del survey (float)
- BRG: rumbo anotado en el registro del survey (float)
- DIP: buzamiento anotado en el registro del survey (float)

#### Comprueba si collar y survey contienen los mismos taladros (BHID)

In [ ]:
at_collar = collar.groupby("BHID")["AT"].max()

In [ ]:
at_survey = survey.groupby(["BHID"])["AT"].max()

In [ ]:
(at_collar == at_survey).sum()

In [ ]:
# Comprobar si la profundidad máxima del collar coincide con la profundidad máxima del survey
(at_collar == at_survey).sum()

In [ ]:
len((at_collar == at_survey))

In [ ]:
survey[["BRG", "DIP"]].describe()

## Table Lito Assay

In [ ]:
table = pd.read_csv("files/dh_raw/Merged_Table_lito_assay.csv")

In [ ]:
table.head()

In [ ]:
table.info()

- holeid : código del taladro (string)
- from, to : profundidad de un tramo del sondaje (float)
- AG_gpt, AS_ppm, AU_gpt, CU_pct, DENSITY, MO_ppm, RECOV, S_pct : columnas de assay (float), contiene valores vacíos
- C_CUATER, R_AGRUP, ROCK, SPLIT : columnas de lito (string), contiene valores vacíos
- mid_x, mid_y, mid_z: valores numéricos (float)

### holeid
#### Comprobar si no existen sondajes adicionales incompletos (no aparecen en algunas de las tablas)

In [ ]:
tableSet = set(table["holeid"].unique())

In [ ]:
collarSet = set(collar["BHID"].unique())

In [ ]:
surveySet = set(survey["BHID"].unique())

In [ ]:
tableSet ^ collarSet

In [ ]:
tableSet ^ surveySet

In [ ]:
collarSet ^ surveySet

### from to

In [ ]:
table["from"]

In [ ]:
((table["from"] - table["to"]) < 0).sum()

#### Verificar que los valores de from y to coincidan numéricamente

In [ ]:
for dh in table["holeid"].unique():
    df = table[table["holeid"] == dh][["from", "to"]]
    df_from = df.iloc[1:, 0].values
    df_to = df.iloc[:-1, 1].values
    if (df_to - df_from).sum() != 0:
        print(f"Los valores de from to en el sondaje {dh} no coinciden.")

### AG_gpt
#### Cuantos valores vacíos hay por cada sondaje

In [ ]:
table[table["holeid"] == "WP001"]["AG_gpt"].isna().sum()

In [ ]:
columna = "AG_gpt"
for dh in table["holeid"].unique():
    df = table[table["holeid"] == dh][columna]
    resultado = df.isna().sum()
    total = len(df)
    if resultado != 0:
        print(f"{dh} - {columna}: {resultado/total:.0%} de valores vacíos")

- En el sondaje WP036, WP057 y WP058, la columna de AG_gpt está vacía.

In [ ]:
def evaluar_vacios(columna):
    print("")
    for dh in table["holeid"].unique():
        df = table[table["holeid"] == dh][columna]
        resultado = df.isna().sum()
        total = len(df)
        if resultado != 0:
            print(f"{dh} - {columna}: {resultado/total:.1%} de valores vacíos")

In [ ]:
table.columns

In [ ]:
import ipywidgets as widgets

In [ ]:
list(table.columns)

In [ ]:
# Evaluar valores de vacios en sondajes de manera automatizada
widgets.interact(evaluar_vacios, columna=list(table.columns), clear_output=True);

### SONDAJE WP036

In [ ]:
len(table[table["holeid"] == "WP036"])

In [ ]:
table[table["holeid"] == "WP036"].isna().sum()

In [ ]:
table[table["holeid"] == "WP057"].isna().sum()

In [ ]:
table[table["holeid"] == "WP058"].isna().sum()

Los sondajes WP036, WP057 y WP058 tienen una mayor cantidad de valores vacíos comparado con el resto de sondajes

### Separar en lito y assay la tabla original

In [ ]:
columns = ["holeid", "from", "to", "AG_gpt", "AS_ppm", "AU_gpt", "CU_pct", "DENSITY", "MO_ppm", "RECOV", "S_pct"]
assay = table[columns].copy()

In [ ]:
assay.rename(columns={"holeid": "ID", "from": "FROM", "to": "TO"}, inplace=True)

In [ ]:
assay.to_csv("files/dh_clean/assay.csv", index=False)

In [ ]:
columns = ["holeid", "from", "to", "C_CUATER", "R_AGRUP", "ROCK", "SPLIT"]
lito = table[columns]

In [ ]:
lito.to_csv("files/dh_clean/lito.csv", index=False)

In [ ]:
collar.to_csv("files/dh_clean/collar.csv", index=False)

In [ ]:
survey.to_csv("files/dh_clean/survey.csv", index=False)

### Guardar y cargar de una database

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [ ]:
engine = create_engine("sqlite:///database.db")

In [ ]:
assay.to_sql(name="table_assay", con=engine)

In [ ]:
columns = ["id"] + list(assay.columns)

In [ ]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM table_assay"))
    result = pd.DataFrame(result, columns=columns)

In [ ]:
result.drop(columns=["id"], inplace=True)

In [ ]:
result

In [ ]:
pd.read_sql("table_assay", con=engine)

### Visualizar sondajes en 3D

In [ ]:
from sondaje3d import DrillData

In [ ]:
collar.rename(columns={"BHID": "ID", "XCOLLAR": "X", "YCOLLAR": "Y", "ZCOLLAR": "Z"}, inplace=True)
collar.drop(columns=["AT"], inplace=True)
survey.rename(columns={"BHID": "ID", "BRG": "AZ"}, inplace=True)

In [ ]:
data = DrillData(collar=collar, survey=survey, table=assay, table_name="Assay")

In [ ]:
data.validate()

In [ ]:
data.table.columns

In [ ]:
data.get_points("S_pct", "numeric")

### Visualizar el valor de AG_gpt en categorías

In [ ]:
data.table["AG_gpt"].describe()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
table = data.table
ag_250 = table[table["AG_gpt"] <= 51]["AG_gpt"]

# Figura principal
fig, ax = plt.subplots(figsize=(12, 6))

# Histograma
ax.hist(ag_250, bins=20)

# Título
ax.set_title("Histograma de Ag (g/t)", fontsize=20)

# Etiquetas de los eje
ax.set_xlabel("Ag (g/t)", fontsize=15)
ax.set_ylabel("Frecuencia", fontsize=15)

# Cambiar escala del eje Y a logarítmico
ax.set_yscale("log")

# Agregar una grilla
ax.grid()

# Mostrar el gráfico
plt.show()

In [ ]:
import numpy as np

def categorizar_AG(row):
    if 0 <= row <= 10:
        return "0 - 10 g/t"
    elif 10 < row <= 50:
        return "10 - 50 g/t"
    elif row > 50:
        return "> 50 g/t"

In [ ]:
data.table["AG_cat"] = data.table["AG_gpt"].apply(categorizar_AG)

In [ ]:
data.table

In [ ]:
data.get_points("AG_cat", "categoric")

In [ ]:
data.plot_3d()

#### Automatizar el gráfico de un histograma por cada columna en Assay

In [ ]:
def histograma(columna):
    # Datos
    df = table[columna]
    
    # Figura principal
    fig, ax = plt.subplots(figsize=(12, 6))

    # Histograma
    ax.hist(df, bins=15)

    # Título
    ax.set_title(f"Histograma de {columna}", fontsize=20)

    # Etiquetas de los eje
    ax.set_xlabel(f"{columna}", fontsize=15)
    ax.set_ylabel("Frecuencia", fontsize=15)

    # Cambiar escala del eje Y a logarítmico
    ax.set_yscale("log")

    # Agregar una grilla
    ax.grid()

    # Mostrar el gráfico
    plt.show()

In [ ]:
widgets.interact(histograma, columna=['AG_gpt', 'AS_ppm', 'AU_gpt', 'CU_pct', 'DENSITY', 'MO_ppm', 'RECOV', 'S_pct']);

In [ ]:
def categorizar_AU(row):
    if 0 <= row <= 1:
        return "0 - 1 g/t"
    elif 1 < row <= 5:
        return "1 - 5 g/t"
    elif row > 5:
        return "> 5 g/t"

In [ ]:
data.table["AU_cat"] = data.table["AU_gpt"].apply(categorizar_AU)

In [ ]:
data.get_points("AU_cat", "categoric")

In [ ]:
data.plot_3d()

### Gráfico de dispersión

In [ ]:
def dispersion(columna1, columna2):
    # Datos
    df1 = table[columna1]
    df2 = table[columna2]
    
    # Figura principal
    fig, ax = plt.subplots(figsize=(12, 6))

    # Dispersión
    ax.scatter(df1, df2, s=15, edgecolor="black")

    # Título
    ax.set_title(f"Diagrama de dispersión de {columna1} vs {columna2}", fontsize=20)

    # Etiquetas de los eje
    ax.set_xlabel(f"{columna1}", fontsize=15)
    ax.set_ylabel(f"{columna2}", fontsize=15)
    
    # Escala logarítmica
    #ax.set_xscale("log")
    #ax.set_yscale("log")
    
    # Agregar una grilla
    ax.grid()

    # Mostrar el gráfico
    plt.show()

In [ ]:
dispersion("AU_gpt", "CU_pct")

In [ ]:
widgets.interact(dispersion, columna1=['AG_gpt', 'AS_ppm', 'AU_gpt', 'CU_pct', 'DENSITY', 'MO_ppm', 'RECOV', 'S_pct'],
                 columna2=['AG_gpt', 'AS_ppm', 'AU_gpt', 'CU_pct', 'DENSITY', 'MO_ppm', 'RECOV', 'S_pct']);

### Categorías en lito

In [ ]:
lito.columns

In [ ]:
lito["C_CUATER"].unique()

In [ ]:
lito["R_AGRUP"].unique()

In [ ]:
lito["ROCK"].unique()

In [ ]:
lito["SPLIT"].unique()

### Visualización de sondajes 3D - Litología

In [ ]:
lito = pd.read_csv("files/dh_clean/lito.csv")

In [ ]:
lito.rename(columns={"holeid": "ID", "from": "FROM", "to": "TO"}, inplace=True)

In [ ]:
data = DrillData(collar=collar, survey=survey, table=lito, table_name="Lito")

In [ ]:
data.validate()

In [ ]:
data.get_points("ROCK", "categoric")

In [ ]:
len(lito["ROCK"].unique())

In [ ]:
data.plot_3d()

In [ ]:
assay

In [ ]:
lito

In [ ]:
data = pd.merge(assay, lito)

In [ ]:
df = data[(data["ROCK"] == "E1") | (data["ROCK"] == "E2") | (data["ROCK"] == "E3")]

In [ ]:
df

In [ ]:
def dispersion(columna1, columna2):
    # Datos
    df1 = df[columna1]
    df2 = df[columna2]
    
    # Figura principal
    fig, ax = plt.subplots(figsize=(12, 6))

    # Dispersión
    ax.scatter(df1, df2, s=15, edgecolor="black")

    # Título
    ax.set_title(f"Diagrama de dispersión de {columna1} vs {columna2}", fontsize=20)

    # Etiquetas de los eje
    ax.set_xlabel(f"{columna1}", fontsize=15)
    ax.set_ylabel(f"{columna2}", fontsize=15)
    
    # Escala logarítmica
    #ax.set_xscale("log")
    ax.set_yscale("log")
    
    # Agregar una grilla
    ax.grid()

    # Mostrar el gráfico
    plt.show()

In [ ]:
widgets.interact(dispersion,
                 columna1=['AG_gpt', 'AS_ppm', 'AU_gpt', 'CU_pct', 'DENSITY', 'MO_ppm', 'RECOV', 'S_pct'],
                 columna2=['AG_gpt', 'AS_ppm', 'AU_gpt', 'CU_pct', 'DENSITY', 'MO_ppm', 'RECOV', 'S_pct']);